# Summarization

https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html

For a more in depth explanation of what these chain types are, see here.
- https://docs.langchain.com/docs/components/chains/index_related_chains

# Index-related chains
이 범주의 체인은 인덱스와 상호 작용하는 데 사용됩니다. 이 체인의 목적은 인덱스에 저장된 자신의 데이터를 LLM과 결합하는 것입니다. 가장 좋은 예는 자신의 문서에 대한 질문 답변입니다.

여기에는 여러 문서를 언어 모델에 전달하는 방법을 이해하는 것이 큰 부분을 차지합니다. 이를 위한 몇 가지 다른 방법, 즉 체인이 있습니다. LangChain은 가장 일반적인 네 가지 방법을 지원하며, 더 많은 방법을 추가하기 위해 적극적으로 노력하고 있으니 아이디어가 있으시면 연락주세요! 어떤 방법을 사용할지 결정하는 것은 상황에 따라 매우 달라질 수 있습니다. 가장 간단한 것부터 가장 복잡한 것까지 순서대로 설명합니다:

## stuff
스터핑은 가장 간단한 방법으로, 모든 관련 데이터를 프롬프트에 컨텍스트로 채워 언어 모델에 전달하기만 하면 됩니다. 이것은 LangChain에서 StuffDocumentsChain으로 구현됩니다.

장점: LLM을 한 번만 호출합니다. 텍스트를 생성할 때 LLM이 모든 데이터에 한 번에 액세스할 수 있습니다.

단점: 대부분의 LLM에는 컨텍스트 길이가 있으며, 큰 문서(또는 많은 문서)의 경우 컨텍스트 길이보다 큰 프롬프트가 표시되므로 이 방법은 작동하지 않습니다.

이 방법의 가장 큰 단점은 작은 데이터 조각에서만 작동한다는 것입니다. 많은 데이터 조각으로 작업하는 경우에는 이 접근 방식을 더 이상 사용할 수 없습니다. 다음 두 가지 접근 방식은 이러한 문제를 해결하기 위해 고안되었습니다.

## map_reduce
이 방법은 각 데이터 청크에 대해 초기 프롬프트를 실행하는 것입니다(요약 작업의 경우 해당 청크에 대한 요약이 될 수 있고, 질문 답변 작업의 경우 해당 청크에만 기반한 답변이 될 수 있습니다). 그런 다음 다른 프롬프트가 실행되어 모든 초기 출력을 결합합니다. 이것은 LangChain에서 MapReduceDocumentsChain으로 구현됩니다.

장점: StuffDocumentsChain보다 더 큰 문서(및 더 많은 문서)로 확장할 수 있습니다. 개별 문서에서 LLM에 대한 호출은 독립적이므로 병렬화할 수 있습니다.

단점: StuffDocumentsChain보다 훨씬 더 많은 LLM 호출이 필요합니다. 최종 결합 호출 중에 일부 정보가 손실됩니다.

## refine
이 방법은 첫 번째 데이터 청크에서 초기 프롬프트를 실행하여 일부 출력을 생성합니다. 나머지 문서의 경우, 해당 출력이 다음 문서와 함께 전달되어 새 문서를 기반으로 출력을 수정하도록 LLM에 요청합니다.

장점: 더 관련성 높은 컨텍스트를 가져올 수 있으며, MapReduceDocumentsChain보다 손실이 적을 수 있습니다.

단점: StuffDocumentsChain보다 LLM에 대한 호출이 더 많이 필요합니다. 또한 호출은 독립적이지 않으므로 MapReduceDocumentsChain처럼 병렬로 실행할 수 없습니다. 또한 문서 순서에 대한 잠재적인 종속성이 있습니다.

## Map-Rerank

이 방법은 각 데이터 청크에 대해 초기 프롬프트를 실행하여 작업을 완료하려고 시도할 뿐만 아니라 답변이 얼마나 확실한지에 대한 점수를 부여하는 것을 포함합니다. 그런 다음 이 점수에 따라 응답의 순위를 매기고 가장 높은 점수를 반환합니다.

장점: MapReduceDocumentsChain과 비슷한 장점. MapReduceDocumentsChain에 비해 더 적은 호출이 필요합니다.

단점: 문서 간 정보를 결합할 수 없습니다. 즉, 하나의 문서에 하나의 간단한 답변이 있을 것으로 예상할 때 가장 유용합니다.


In [10]:
from pprint import pprint
with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')
pprint(transcript[:200])


('인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n'
 '\n'
 '숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n'
 '\n'
 '이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n'
 '\n'
 'NVIDIA GPU도 완벽한 칩은 아니에요.\n'
 '\n'
 '예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n'
 '\n'
 '근데 그렇게')


In [11]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

llm = OpenAI(temperature=0)
text_splitter = CharacterTextSplitter()

In [19]:
from langchain.docstore.document import Document
# def __init__(self, separator: str = "\n\n", **kwargs: Any):
#         """Create a new TextSplitter."""
#         super().__init__(**kwargs)
#         self._separator = separator
# split_text 함수를 들어가 보면 \n\n을 기준으로 split을 하고 있음
texts = text_splitter.split_text(transcript)
docs = [Document(page_content=t) for t in texts]

In [37]:
# docs 하나당 몇 토큰인지 알아보자
from transformers import AutoTokenizer
import tiktoken

electra_tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-discriminator")
tokenized_electra = electra_tokenizer.encode(transcript)

enc = tiktoken.encoding_for_model("text-davinci-003")
tokenized_text_davinci003 = enc.encode(transcript)

enc35 = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokenized_text_chatgpt35 = enc35.encode(transcript)

print(len(tokenized_electra)) # electra : 4380
print(len(tokenized_text_davinci003)) # davinci : 23071
print(len(tokenized_text_chatgpt35)) # chatgpt : 10574


Token indices sequence length is longer than the specified maximum sequence length for this model (4380 > 512). Running this sequence through the model will result in indexing errors


4380
23071
10574


In [44]:
pprint(docs)

for i in range(3):
    print(len(enc35.encode(docs[i].page_content))) 
    # text splitter를 적용된 문서들의 토큰수를 chatgpt3.5 기준으로 확인해보면 4000토큰 미만으로 나누어짐
    # 추가적으로 나누어진 텍스트를 보면 이전 doc의 마지막 문장을 어느정도 포함하고 있음

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

In [52]:
# 200 미만으로 문장을 chunk_overlap 함
'''
class TextSplitter(ABC):
    """Interface for splitting text into chunks."""

    def __init__(
        self,
        chunk_size: int = 4000,
        chunk_overlap: int = 200,
        length_function: Callable[[str], int] = len,
    ):
        """Create a new TextSplitter."""
        if chunk_overlap > chunk_size:
            raise ValueError(
                f"Got a larger chunk overlap ({chunk_overlap}) than chunk size "
                f"({chunk_size}), should be smaller."
            )
        self._chunk_size = chunk_size
        self._chunk_overlap = chunk_overlap
        self._length_function = length_function
'''

docs[0].page_content.find('그 문제가')
docs[0].page_content[3819:]
len(docs[0].page_content[3819:]) # 118
docs[1].page_content.find('그러니까 이런 느낌이에요')
len(docs[1].page_content[3797:]) # 191

118


191

In [4]:
# refine
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from pprint import pprint
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')

text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_text(transcript) # 문장 분리 길이 4000 미만 및 chunk_overlap 200
docs = [Document(page_content=t) for t in texts]
pprint(docs)

############################################################################################################
bullet_point_prompt_template = """다음 문장을 요약해주세요:

{text}

한국어 문장요약 결과:
"""
############################################################################################################
bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])

llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llmc, chain_type="refine", return_intermediate_steps=True, refine_prompt=bullet_point_prompt, verbose=False)
# chain = load_summarize_chain(llmc, chain_type="refine", return_intermediate_steps=True, verbose=True)

res = chain({"input_documents": docs}, return_only_outputs=True)
res

# Load_summarize_chain 함수를 들어가보면 refine_prompt가 있음 그리고 refine_prompt는 PromptTemplate 이 내부적으로 있는데 
# documents가 여러개라고 했을때 첫번때 doc은 영어로된 프롬프트 먼저 받고 2번째 부터 내가 넣어준 템플릿을 따르는 경향이 있음.. 이거 에러 아닌가?

'''
issue: 
fork and pull request로 기여해보자
To contribute to this project, please follow a "fork and pull request" workflow. Please do not try to push directly to this repo unless you are maintainer.
https://github.com/hwchase17/langchain/blob/master/.github/CONTRIBUTING.md

요약 체인에서 제가 정의한 프롬프트로 요약을 진행하려고 하면, 먼저 사전에 정의된 refine.py 의 프롬프트가 1차적으로 실행된 뒤 제가 정의한 프롬프트가 2번째 부터 적용됩니다. 
When I try to run the summarization with the prompts I defined in the summarization chain, the predefined prompts from refine.py are executed first, and then the prompts I defined are applied second. 
'''

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

{'intermediate_steps': ['인공신경망은 CPU와는 다른 역할을 수행하며, 데이터 입력만으로 학습되어 용량이 커집니다. 인공지능 기술 개발에는 SVM 등 다양한 아이디어가 사용되며, 인공신경망은 인간의 뇌세포 구조를 모방하면서 학습 시킬 수 있는 방법론 중 하나입니다. CPU는 조건을 잘 처리하는데, GPU는 숫자 계산을 잘합니다.',
  'AI 기술은 CPU가 좀 더 똑똑한 칩인데, 현재는 무식한 방식으로 좀 더 나은 지성이 구현된 상태이다. 인공신경망은 CPU에 비해 더 좋은 성능을 보이는 GPU를 사용하여 계산한다. 하지만 GPU를 사용하는 것은 CPU를 사용하는 것보다 진입장벽이 높기 때문에 쓰는 방법을 파악하는 것이 중요하다. 인공신경망을 만들기 위해서는 뇌세포와 뇌세포의 연결관계를 고려해야 하기 때문에 연구는 어렵지만, GPU와 연결하면 좋은 성능을 보인다.',
  '엔비디아의 그래픽 처리 장치(GPU)가 인공지능, 슈퍼컴퓨터 등에 사용되어 성능이 크게 개선되었고, 이는 생태계를 형성하는 반도체 비즈니스에서 중요한 요소가 되었다. 메모리 중에서도 고용량 고속 메모리(HBM) 수요가 높아진 상황이며, 한국에서도 인공지능 분야에서 HBM 등이 많이 사용된다. 하지만, 단순한 연구개발 수요보다는 꾸준한 사용 수요가 필요하며, 이는 연구를 통해 만들어야 한다는 전문가의 견해도 있다.',
  'GPT와 같은 인공지능 메모리 시장은 꾸준한 사용 수요가 필요하며, 따라서 데이터 쌍이 잘 갖춰져 있는 곳에서 조용히 발전해야 한다. 현재 인공지능은 데이터 쌍이 필요한데 인터넷에서 적합한 데이터를 찾기 어렵고, 그 중에서도 신뢰할 수 있는 말이 있는지가 중요하다. GPT는 요약 작업에 유용하기 때문에 이미 인터넷에 많은 요약 데이터가 있으며, 이 데이터를 이용해 학습할 수 있다. 하지만 GPT가 사용되는 사업 분야가 다양해질수록 많은 데이터와 더 나은 반도체가 필요하게 된다.'],
 'output_text': 'GPT와 같은 인공지능 메모리 시장은 꾸

In [5]:
pprint(res['output_text'])

('GPT와 같은 인공지능 메모리 시장은 꾸준한 사용 수요가 필요하며, 따라서 데이터 쌍이 잘 갖춰져 있는 곳에서 조용히 발전해야 한다. '
 '현재 인공지능은 데이터 쌍이 필요한데 인터넷에서 적합한 데이터를 찾기 어렵고, 그 중에서도 신뢰할 수 있는 말이 있는지가 중요하다. '
 'GPT는 요약 작업에 유용하기 때문에 이미 인터넷에 많은 요약 데이터가 있으며, 이 데이터를 이용해 학습할 수 있다. 하지만 GPT가 '
 '사용되는 사업 분야가 다양해질수록 많은 데이터와 더 나은 반도체가 필요하게 된다.')


---
# The `stuff` chain

In [14]:
# stuff chain은 한번에 통째로 입력을 집어넣어서 요약을 진행함으로 문장을 모델이 감당할 수 있을 정도로 미리 정의해서 넣어줘야함
# 그리고 아래 출력은 넘어갔을때 에러를 일부로 보여주는 예시임

# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from pprint import pprint
import tiktoken
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')
enc35 = tiktoken.encoding_for_model("gpt-3.5-turbo")
# tokenized_text_chatgpt35 = enc35.encode(transcript)
# print(len(tokenized_text_chatgpt35))

text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_text(transcript) # 문장 분리 길이 4000 미만 및 chunk_overlap 200
docs = [Document(page_content=t) for t in texts]
pprint(docs)
tmp_tok = 0
for doc in docs:
    tokenized_text_chatgpt35 = enc35.encode(doc.page_content)
    # print(len(tokenized_text_chatgpt35))
    tmp_tok += len(tokenized_text_chatgpt35)
print('전체 토큰수', tmp_tok)

############################################################################################################
bullet_point_prompt_template = """다음 문장을 요약해주세요:

{text}

한국어 문장요약 결과:
"""
############################################################################################################
bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])

# 모든 데이터에 한번에 엑세스 하므로 전체 토큰이 
llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llmc, 
                             chain_type="stuff",  
                             prompt=bullet_point_prompt)
'''
스터핑은 가장 간단한 방법으로, 모든 관련 데이터를 프롬프트에 컨텍스트로 채워 언어 모델에 전달하기만 하면 됩니다. 이것은 LangChain에서 StuffDocumentsChain으로 구현됩니다.

장점: LLM을 한 번만 호출합니다. 텍스트를 생성할 때 LLM이 모든 데이터에 한 번에 액세스할 수 있습니다.

단점: 대부분의 LLM에는 컨텍스트 길이가 있으며, 큰 문서(또는 많은 문서)의 경우 컨텍스트 길이보다 큰 프롬프트가 표시되므로 이 방법은 작동하지 않습니다.

이 방법의 가장 큰 단점은 작은 데이터 조각에서만 작동한다는 것입니다. 많은 데이터 조각으로 작업하는 경우에는 이 접근 방식을 더 이상 사용할 수 없습니다. 다음 두 가지 접근 방식은 이러한 문제를 해결하기 위해 고안되었습니다.
'''

# res = chain({"input_documents": docs}, return_only_outputs=True)
chain.run(docs) # 이렇게 stuff는 한번에 언어모델이 감당할 수 있는 토큰수를 넘어가면 안됨
res

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 11128 tokens. Please reduce the length of the messages.

In [18]:
docs[0].page_content

'인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인공지능은 어떤 물건이냐 하면요.\n\n스스

In [61]:
# refine
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from pprint import pprint
import tiktoken
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')
enc35 = tiktoken.encoding_for_model("gpt-3.5-turbo")
# tokenized_text_chatgpt35 = enc35.encode(transcript)
# print(len(tokenized_text_chatgpt35))

text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_text(transcript) # 문장 분리 길이 4000 미만 및 chunk_overlap 200
docs = [Document(page_content=t) for t in texts]
pprint(docs)

# 나뉜 문단별 토큰수 계산
tmp_tok = 0
for doc in docs:
    tokenized_text_chatgpt35 = enc35.encode(doc.page_content)
    # print(len(tokenized_text_chatgpt35))
    tmp_tok += len(tokenized_text_chatgpt35)
print('전체 토큰수', tmp_tok)

# 템플릿 작성
############################################################################################################
bullet_point_prompt_template = """다음 문장을 요약해주세요:

{text}

한국어 문장요약 결과:
"""
############################################################################################################
bullet_point_prompt = PromptTemplate(template=bullet_point_prompt_template, input_variables=["text"])

# 모든 데이터에 한번에 엑세스 하므로 전체 토큰이 
llmc = ChatOpenAI(temperature=0.9, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llmc, 
                             chain_type="stuff",  
                             prompt=bullet_point_prompt,
                             verbose=True)
'''
스터핑은 가장 간단한 방법으로, 모든 관련 데이터를 프롬프트에 컨텍스트로 채워 언어 모델에 전달하기만 하면 됩니다. 이것은 LangChain에서 StuffDocumentsChain으로 구현됩니다.

장점: LLM을 한 번만 호출합니다. 텍스트를 생성할 때 LLM이 모든 데이터에 한 번에 액세스할 수 있습니다.

단점: 대부분의 LLM에는 컨텍스트 길이가 있으며, 큰 문서(또는 많은 문서)의 경우 컨텍스트 길이보다 큰 프롬프트가 표시되므로 이 방법은 작동하지 않습니다.

이 방법의 가장 큰 단점은 작은 데이터 조각에서만 작동한다는 것입니다. 많은 데이터 조각으로 작업하는 경우에는 이 접근 방식을 더 이상 사용할 수 없습니다. 다음 두 가지 접근 방식은 이러한 문제를 해결하기 위해 고안되었습니다.
'''
 # 이렇게 stuff는 한번에 언어모델이 감당할 수 있는 토큰수를 넘어가면 안되고 한번에 안넘어가는 토큰수에 한해서 넣어줘야함
chain.run(docs[:1])

# 내가 생각하는 stuff의 장점은 
# 한번에 짧은 문단 그치만 다른 주제에 대해서 여러 문단이 있는경우 
# Stuff로 한번에 하나씩 요약을 미리 해두는 기능으로 사용하면 될듯
# 그리고 그 요약된 문단을 다시 모아서 한번에 요약하는 방식으로 사용하면 좋을듯

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

'인공신경망은 CPU가 잘하는 일을 잘하지 않는 특징이 있으며, 인공지능 개발에는 데이터가 필요하다. 인공신경망은 CPU 대신 NVIDIA GPU를 사용하며, 인공지능 기술의 핵심인 인공신경망은 스스로 학습할 수 있는 능력을 가진다. CPU는 조건 파악을 잘하며, GPU는 대량의 숫자 계산을 잘한다. 프로그램의 조건이 중요한 경우 CPU가 유리하며, 인공신경망의 경우 GPU가 더 효율적이다.'

In [59]:
# 영어로 요약해보기
with open("./state_of_the_union.txt") as f:
    state_of_the_union = f.read()
stu_text = text_splitter.split_text(state_of_the_union) # list 형태로 split 한다

stu_docs = [Document(page_content=t) for t in stu_text[:3]]
stu_docs

llm = OpenAI(temperature=0)

chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(stu_docs[:2])

" In this speech, the speaker addresses the American people and encourages them to stand with the Ukrainian people in their struggle against Russia's aggression. The speaker emphasizes the courage and determination of the Ukrainian people and the need for dictators to be held accountable for their actions. The speaker calls for the United States to send a signal of support to Ukraine and the world."

---
# The `map_reduce` Chain

In [ ]:
# /autogpt/lib/python3.9/site-packages/langchain/chains/summarize/map_reduce_prompt.py
# map_reduce chain 이렇게 생김
# flake8: noqa
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

In [65]:
# refine
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from pprint import pprint
import tiktoken
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')
enc35 = tiktoken.encoding_for_model("gpt-3.5-turbo")
# tokenized_text_chatgpt35 = enc35.encode(transcript)
# print(len(tokenized_text_chatgpt35))

text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_text(transcript) # 문장 분리 길이 4000 미만 및 chunk_overlap 200
docs = [Document(page_content=t) for t in texts]
pprint(docs)

# 나뉜 문단별 토큰수 계산
tmp_tok = 0
for doc in docs:
    tokenized_text_chatgpt35 = enc35.encode(doc.page_content)
    # print(len(tokenized_text_chatgpt35))
    tmp_tok += len(tokenized_text_chatgpt35)
print('전체 토큰수', tmp_tok)

# 템플릿 작성
############################################################################################################
prompt_template = """다음 문장을 요약해주세요:

{text}

한국어 문장요약 결과:
"""
############################################################################################################
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

# 모든 데이터에 한번에 엑세스 하므로 전체 토큰이 
llmc = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") # chatgpt 3.5 model
chain = load_summarize_chain(llmc, 
                             chain_type="map_reduce",  # map_reduce 방식 각각의 chunk에 대해 요약을 먼저 진행하고 그 결과를 다시 요약하는 방식 -> 각각의 요약에대해 병렬처리함
                             return_intermediate_steps=True,
                             map_prompt=PROMPT,
                             combine_prompt=PROMPT,
                             verbose=True)
'''
map_reduce 방법은 각 데이터 청크에 대해 초기 프롬프트를 실행하는 것입니다
(요약 작업의 경우 해당 청크에 대한 요약이 될 수 있고, 질문 답변 작업의 경우 해당 청크에만 기반한 답변이 될 수 있습니다).

그런 다음 다른 프롬프트가 실행되어 모든 초기 출력을 결합합니다. 이것은 LangChain에서 MapReduceDocumentsChain으로 구현됩니다.

장점: StuffDocumentsChain보다 더 큰 문서(및 더 많은 문서)로 확장할 수 있습니다.
     개별 문서에서 LLM에 대한 호출은 독립적이므로 병렬화할 수 있습니다.

단점: StuffDocumentsChain보다 훨씬 더 많은 LLM 호출이 필요합니다. 최종 결합 호출 중에 일부 정보가 손실됩니다.
'''
# chain.run(docs[:])
res = chain({"input_documents": docs}, return_only_outputs=True)

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

In [67]:
pprint(res)

{'intermediate_steps': ['인공신경망은 CPU가 잘하는 일이 아니며, 인공지능 개발에는 데이터가 필요하다. 인공신경망은 '
                        '용량이 크고, NVIDIA GPU도 완벽한 칩은 아니며, 인공지능을 만들기 위해서는 여러 가지 '
                        '아이디어를 사용한다. 인공신경망은 CPU보다는 GPU를 사용하며, 인공신경망은 입력값을 '
                        '소수점으로 바꾼 다음 소수점 곱셈을 엄청나게 한다.',
                        '인공지능 기술은 CPU가 더 똑똑한 칩인데, 현재 인공지능 기술은 무식한 방식으로 구현되어 '
                        '있다. 인공신경망을 만들기 위해서는 뇌세포와 뇌세포의 연결관계를 어떻게 해줘야 하는지를 알아내야 '
                        '하며, GPU를 사용하여 그래픽과 비슷한 연산을 요구하는 인공지능 기술을 구현하고 있다. 하지만 '
                        'GPU를 사용하는 방법을 모르는 사람들이 많아서 진입장벽이 높다.',
                        '엔비디아의 그래픽 카드는 슈퍼컴퓨터나 시뮬레이션 등 물리 연산에도 사용되며, CUDA 등의 '
                        '기술로 GPU용 프로그램 개발을 지원한다. 이를 활용한 인공지능 연구개발 수요는 크겠지만, '
                        '꾸준한 사용 수요를 만들기 위해서는 연구가 필요하다. 메모리 중 고용량 고속 메모리가 '
                        '인공신경망에서 중요하게 사용되며, 한국에서는 이에 대한 수요가 크다. 하지만 메모리 용량이 크기 '
                        '때문에 물량은 많지 않다. 이에 대한 해결책으로는 GPT 등의 기술이 있다.',
                       

---
# The `refine` Chain

In [ ]:
# refine chain 이렇게 생김
# 프롬프트 하나 때리고 나서 refine 프롬프트를 또 때림

# /autogpt/lib/python3.9/site-packages/langchain/chains/summarize/refine_prompts.py
# flake8: noqa
from langchain.prompts import PromptTemplate

REFINE_PROMPT_TMPL = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
REFINE_PROMPT = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=REFINE_PROMPT_TMPL,
)


prompt_template = """Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])



In [71]:
# refine
# https://python.langchain.com/en/latest/reference/modules/text_splitter.html
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from pprint import pprint
import tiktoken
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


with open('./정인성작가_transcript.txt', 'r', encoding='utf-8') as f:
    transcript = f.read()

transcript = transcript.replace('. ', '.\n\n')
enc35 = tiktoken.encoding_for_model("gpt-3.5-turbo")
# tokenized_text_chatgpt35 = enc35.encode(transcript)
# print(len(tokenized_text_chatgpt35))

text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=200)
texts = text_splitter.split_text(transcript) # 문장 분리 길이 4000 미만 및 chunk_overlap 200
docs = [Document(page_content=t) for t in texts]
pprint(docs)

# 나뉜 문단별 토큰수 계산
tmp_tok = 0
for doc in docs:
    tokenized_text_chatgpt35 = enc35.encode(doc.page_content)
    # print(len(tokenized_text_chatgpt35))
    tmp_tok += len(tokenized_text_chatgpt35)
print('전체 토큰수', tmp_tok)

# 템플릿 작성
############################################################################################################
prompt_template = """다음 문장을 요약해주세요:

{text}

한국어 문장요약 결과:
"""
############################################################################################################
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

refine_template = (
    "당신의 임무는 최종 요약을 작성하는 것입니다\n"
    "특정 지점까지 기존 요약을 제공했습니다: {existing_answer}\n"
    "기존 요약을 다듬을 기회가 있습니다."
    "(필요한 경우에만) 아래에 더 많은 컨텍스트를 추가할 수 있습니다.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "새로운 문맥이 주어지면, 원래 요약을 한국어로 수정하세요."
    "문맥이 유용하지 않은 경우 원래 요약을 반환합니다."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

# 모든 데이터에 한번에 엑세스 하므로 전체 토큰이 
llmc = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") # chatgpt 3.5 model
chain = load_summarize_chain(llmc, 
                             chain_type="refine",  # refine 방식
                             return_intermediate_steps=True,
                             question_prompt=PROMPT,
                             refine_prompt=refine_prompt,
                             verbose=True)
'''
이 방법은 첫 번째 데이터 청크에서 초기 프롬프트를 실행하여 일부 출력을 생성합니다. 
나머지 문서의 경우, 해당 출력이 다음 문서와 함께 전달되어 새 문서를 기반으로 출력을 수정하도록 LLM에 요청합니다.

장점: 더 관련성 높은 컨텍스트를 가져올 수 있으며, MapReduceDocumentsChain보다 손실이 적을 수 있습니다.

단점: StuffDocumentsChain보다 LLM에 대한 호출이 더 많이 필요합니다. 
     또한 호출은 독립적이지 않으므로 MapReduceDocumentsChain처럼 병렬로 실행할 수 없습니다.
     또한 문서 순서에 대한 잠재적인 종속성이 있습니다.
'''
# chain.run(docs[:])
res = chain({"input_documents": docs}, return_only_outputs=True)

# refine chain 이렇게 생김
# 프롬프트 하나 때리고 나서 refine 프롬프트를 또 때림 
# verbose 기능으로 보면 알 수 있다
# 
# issue: 아래처럼 issue인줄 알았는데 기능이었다 ㅋㅋㅋㅋㅋㅋ 이렇게 되는게 싫으면 chain_type="map_reduce"로 하면 된다
# fork and pull request로 기여해보자
# To contribute to this project, please follow a "fork and pull request" workflow. Please do not try to push directly to this repo unless you are maintainer.
# https://github.com/hwchase17/langchain/blob/master/.github/CONTRIBUTING.md

# 요약 체인에서 제가 정의한 프롬프트로 요약을 진행하려고 하면, 먼저 사전에 정의된 refine.py 의 프롬프트가 1차적으로 실행된 뒤 제가 정의한 프롬프트가 2번째 부터 적용됩니다. 
# When I try to run the summarization with the prompts I defined in the summarization chain, the predefined prompts from refine.py are executed first, and then the prompts I defined are applied second. 
# 

[Document(page_content='인공신경망은 특징이 CPU가 잘하는 일은 아니라는 거예요.\n\n숫자가 5를 넘으면 안녕하세요 말하는 코드를 쓴다.\n\n이거는 용량 진짜로 몇 킬로바이트 안 하거든요? 근데 제가 동일한 일을 하려는 걸 인공신경망으로 만들면 용량이 몇백배 커요.\n\nNVIDIA GPU도 완벽한 칩은 아니에요.\n\n예전 슈퍼컴퓨터에 그래픽카드 만 개를 엮었다고 했잖아요.\n\n근데 그렇게 엮으면 비효율도 크고 비용도 크거든요.\n\nGPT를 통한 인공지능 변호사 이런 거 만들려면 데이터 장벽부터 있는 거예요.\n\n예전에는 PC였고 스마트폰이었고 지금은 서버인데 제 사회의 어떤 수요를 만들려면요.\n\n안녕하세요.\n\n머니인사이드 시청자 여러분 정인성 작가입니다.\n\n저는 원래 반도체 회사에서 반도체 관련한 시뮬레이션 일을 하고 있었고요.\n\n최근에 인공지능 관련해서도 새로운 책을 쓸 수 있었고요.\n\n지금은 이제 동료들이랑 인공지능 개발 쪽으로 와 있습니다.\n\n스타릿에서 어떤...\n\n반도체들을 완제품을 만들고 나면 이 반도체들을 또 뭉쳐서 그 위에다가 조그마한 컨트롤러를 붙여서 막 동작을 하게 만들어요.\n\n쉽게 말하면 원본 반도체가 아니고 반도체들을 결합해서 새로운 가치를 만드는 거죠.\n\n이제 그런 것들을 시뮬레이션으로 검증하는 그런 일들을 했죠.\n\n인공지능은 용어를 구분할 필요가 있는데 인공지능은 우리 꿈 같은 거예요.\n\n그러니까 우리가 하고 싶은 거 쉽게 말하면 이 인공지능이라는 요소가 딱 잘 정의되진 않는데 예를 들면 예전에 있던 그 바둑에 있는 바둑 AI나 그 스타크래프트에서 이제 컴퓨터 상대를 넣었어요.\n\n그런 것도 다 인공지능이라고 불러요.\n\n실제로는 그렇게 부를 수 있고요.\n\n그러니까 인공지능이라는 것 자체가 굉장히 어마어마한 개념이고 그렇진 않은 거예요.\n\n근데 이제 우리 일반인들 입장에서는 좀 터미네이터 같은 걸 생각하겠죠.\n\n인공지능으로.\n\n저 일하는 입장에서 인

In [72]:
res

{'intermediate_steps': ['인공신경망은 CPU가 잘하는 일이 아니며, 인공지능 개발에는 데이터가 필요하다. 인공신경망은 용량이 크고, NVIDIA GPU도 완벽한 칩은 아니며, 인공지능을 만들기 위해서는 여러 가지 아이디어를 사용한다. 인공신경망은 CPU보다는 GPU를 사용하며, 인공신경망은 입력값을 소수점으로 바꾼 다음 소수점 곱셈을 많이 사용한다.',
  '인공신경망은 CPU보다는 GPU를 사용하며, 입력값을 소수점으로 바꾼 다음 소수점 곱셈을 많이 사용한다. 인공지능 개발에는 데이터가 필요하며, 인공신경망은 용량이 크고 NVIDIA GPU도 완벽한 칩은 아니다. 인공지능을 만들기 위해서는 여러 가지 아이디어를 사용한다. 인공신경망은 CPU가 잘하는 일이 아니며, 인공지능 기술은 CPU가 아닌 GPU와 같은 칩을 사용해야 한다. 인공신경망은 인간 뇌세포를 모방한 것이며, 연결관계를 수조개 모아놓으면 지성이 된다. 인공신경망을 만들기 위해서는 GPU를 사용하는 것이 좋으며, GPU를 사용하는 방법을 익히는 것이 중요하다.',
  '인공신경망은 CPU보다는 GPU를 사용하며, 입력값을 소수점으로 바꾼 다음 소수점 곱셈을 많이 사용한다는 것이 기존 요약이었다. 인공지능 개발에는 데이터가 필요하며, 인공신경망은 용량이 크고 NVIDIA GPU도 완벽한 칩은 아니다. 인공신경망은 인간 뇌세포를 모방한 것이며, 연결관계를 수조개 모아놓으면 지성이 된다. 인공신경망을 만들기 위해서는 GPU를 사용하는 것이 좋으며, GPU를 사용하는 방법을 익히는 것이 중요하다는 것이다. 추가적으로, 인공지능을 만들기 위해서는 여러 가지 아이디어를 사용하며, 인공신경망은 CPU가 잘하는 일이 아니며, 인공지능 기술은 CPU가 아닌 GPU와 같은 칩을 사용해야 한다는 것이다. 또한, 메모리 중에 고용량, 고대역폭 메모리가 중요하며, 인공신경망의 용량이 크기 때문에 메모리 용량이 큰 것이 필요하다는 것이다.',
  '인공신경망은 GPU를 사용하며, 데이터가 필요하다. 인공

In [68]:
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

chain.run(stu_docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to e

'\n\n지난해 코로나로 인해 우리는 떨어져 있었습니다. 오늘 우리는 민주당, 공화당, 독립운동가로 모이고 있습니다. 우리는 서로, 미국 사람들, 헌법에 대한 의무를 가지고 있습니다. 러시아의 블라디미르 푸틴이 자유 세계의 '